In [1]:
import pygame
from pygame.locals import *
import time
import requests
import json

pygame 2.4.0 (SDL 2.26.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
SERVER_IP = "http://54.165.59.37:5000"

In [3]:
pygame.init()

(4, 1)

In [4]:
pygame.joystick.init()

In [5]:
controller = pygame.joystick.Joystick(0)

In [6]:
def get_button(colour,controller):
    pygame.event.get()
    if colour.lower() == "r":
        button = 1
    elif colour.lower() == "b":
        button = 2
    elif colour.lower() == "y":
        button = 3
    elif colour.lower() == "a":
        button = 0
    return controller.get_button(button)

In [7]:
get_button("r",controller)

0

In [8]:
def activate_light(colour, state):
    if colour.lower() == "a":
        data = {"energy status":1,"override_red":state,"override_yellow":state,"override_blue":state}
        res = requests.post(url=SERVER_IP+"/led_driver/red",json=data)
        res = requests.post(url=SERVER_IP+"/led_driver/yellow",json=data)
        res = requests.post(url=SERVER_IP+"/led_driver/blue",json=data)
    if colour.lower() == "r":
        data = {"energy status":1,"override_red":state,"override_yellow":state,"override_blue":state}
        res = requests.post(url=SERVER_IP+"/led_driver/red",json=data)
    elif colour.lower() == "b":
        data = {"energy status":1,"override_red":state,"override_yellow":state,"override_blue":state}
        res = requests.post(url=SERVER_IP+"/led_driver/blue",json=data)
    elif colour.lower() == "y":
        data = {"energy status":1,"override_red":state,"override_yellow":state,"override_blue":state}
        res = requests.post(url=SERVER_IP+"/led_driver/yellow",json=data)
    print(f"turned {colour} light {state}")

In [9]:
def get_value(axis,controller):
    pygame.event.get()
    if axis=="x":
        axis = 0
    elif axis=="y":
        axis = 1
    value = controller.get_axis(axis)
    if abs(value)<0.2:
        value = float(0)
    return value

In [12]:
import bluetooth

target_name = "dwayne"
target_address = None

nearby_devices = bluetooth.discover_devices()

for bdaddr in nearby_devices:
    if target_name == bluetooth.lookup_name( bdaddr ):
        target_address = bdaddr
        break

if target_address is not None:
    print("found target bluetooth device with address ", target_address)
else:
    print("could not find target bluetooth device nearby")

found target bluetooth device with address  24:0A:C4:11:0F:A2


In [13]:
nearby_devices

['24:0A:C4:11:0F:A2']

In [14]:
services = bluetooth.find_service(address=target_address)
print(services)


[{'host': '24:0A:C4:11:0F:A2', 'name': b'ESP32SPP\x00', 'description': '', 'port': 1, 'protocol': 'RFCOMM', 'rawrecord': b'6\x00D\t\x00\x00\n\x00\x01\x00\x00\t\x00\x015\x03\x19\x11\x01\t\x00\x045\x0c5\x03\x19\x01\x005\x05\x19\x00\x03\x08\x01\t\x00\x055\x03\x19\x10\x02\t\x00\t5\x085\x06\x19\x11\x01\t\x01\x02\t\x01\x00%\tESP32SPP\x00', 'service-classes': [b'1101'], 'profiles': [(b'1101', 258)], 'provider': None, 'service-id': None, 'handle': 65536}]


In [15]:
host = services[0]['host']
port = services[0]['port']
sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)
sock.connect((host, port))


In [16]:
light_states={"r":0,"b":0,"y":0, "a":0}

In [17]:
while(1):
    time.sleep(0.2)
    xval = -get_value("y",controller)*200
    yval = -get_value("x",controller)*75
    data = f'x{round(xval,1)}y{round(yval,1)}'.encode('utf-8')
    if get_button('r',controller):
        light_states['r'] = not light_states['r']
        try:
            activate_light("r",light_states['r'])
        except:
            continue
    if get_button('b',controller):
        light_states['b'] = not light_states['b']
        try:
            activate_light("b",light_states['b'])
        except:
            continue
    if get_button('y',controller):
        light_states['y'] = not light_states['y']
        try:
            activate_light("y",light_states['y'])
        except:
            continue
    if get_button('a',controller):
        light_states['a'] = not light_states['a']
        try:
            activate_light("a",light_states['a'])
        except:
            continue
    
    if(yval+xval)!=0:
        print(data)
        try:
            sock.send(data)
        except:
            continue
    else:
        print(data)
    

b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x189.1y-44.2'
b'x165.9y-54.4'
b'x173.6y-54.2'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y-0.0'
b'x-0.0y52.2'
b'x-0.0y75.0'
b'x-49.2y75.0'
b'x-49.6y75.0'
b'x-49.8y75.0'
b'x-0.0y75.0'
b'x166.4y46.6'
b'x179.8y43.2'
b'x181.7y40.9'
b'x192.0y29.5'
b'x200.0y-0.0'
b'x200.0y-0.0'
b'x200.0y-22.1'
b'x200.0y-24.2'
b'x200.0y-19.3'
b'x200.0y-0.0'
b'x200.0y23.5'
b'x200.0y24.0'
b'x200.0y21.8'
b'x200.0y22.1'
b'x200.0y18.0'
b'x200.0y17.0'
b'x200.0y-25.8'
b'x193.3y-40.2'
b'x189.4y-44.3'
b'x187.5y-46.0'
b'x187.5y-46.0'
b'x187.5y-46.0'
b'x-157.5y-23.6'
b'x-200.0y-0.0'
b'x-200.0y-0.0'
b'x-200.0y-0.0'
b'x-200.0y-0.0'
b'x-200.0y-0.0'
b'x-200.0y-0.0'
b'x-200.0y-0.0'
b'x-20

OSError: An established connection was aborted by the software in your host machine.


In [ ]:
sock.close()